In [7]:
import pandas as pd
import requests

In [8]:
input_file='../data/estoque.xlsx'

rota '/estoque'

importaÃ§Ã£o da contagem de estoque

In [9]:
estoque = pd.read_excel(input_file).T.set_index(0).dropna(axis=0, how='all').T
estoque.drop(['ESTOQUE'], axis=1, inplace=True)
estoque.columns = ['nome', 'estoque']
estoque['estoque'] = pd.to_numeric(estoque['estoque'], errors='coerce')
estoque.dropna(inplace=True)
#pylint estoque

listagem de produtos tiny API

In [10]:
produtos_id = list()

In [11]:
data = {
    'formato': 'json',
    'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
    'pagina': 1
}

In [12]:
while True:
    try:
        produtos_id += [x['produto'] for x in requests.post(
            'https://api.tiny.com.br/api2/produtos.pesquisa.php', data).json()['retorno']['produtos']]
        data['pagina'] += 1
    except:
        break

In [13]:
produtos_id = pd.DataFrame(produtos_id)
produtos_id = produtos_id.T.reindex(['id', 'nome', 'estoque']).T
produtos_id = estoque.merge(produtos_id)

requisiÃ§Ã£o de dados completos do produto tiny API

In [18]:
a = [{'id': '623449902',
  'nome': 'CORAÇÃO',
  'codigo': 'CORAGAT',
  'unidade': 'kg',
  'estoque_minimo': 6,
  'marca': 'Jerônimos',
  'slug': 'Coração',
  'cotacao': 0,
  'disponivel': False,
  'quantidade': 0,
  'valorfinal': 0},
]

a[0].keys()

dict_keys(['id', 'nome', 'codigo', 'unidade', 'estoque_minimo', 'marca', 'slug', 'cotacao', 'disponivel', 'quantidade', 'valorfinal'])

In [19]:
produtos_completos = [requests.post('https://api.tiny.com.br/api2/produto.obter.php', {
                                    'formato': 'json', 'token': '2b6fc7102240cedcc9166c43921ea73eea82b876', 'id': x}).json()['retorno']['produto'] for x in produtos_id['id']]
produtos_completos = pd.DataFrame(produtos_completos).T.reindex(['id', 'nome', 'codigo', 'unidade', 'estoque_minimo', 'marca', 'slug', 'cotacao', 'disponivel', 'quantidade', 'valorfinal']).T

formataÃ§Ã£o final para /produtos

In [ ]:
produtos_formatados = produtos_completos
produtos_formatados['unidade'] = produtos_formatados['unidade'].str.casefold()
produtos_formatados['unidade'] = [
    'kg' if 'kg' in x else 'un' for x in produtos_formatados['unidade']]
produtos_formatados['cotacao'] = 0
produtos_formatados['disponivel'] = False
produtos_formatados['quantidade'] = 0
produtos_formatados['valorfinal'] = 0

In [ ]:
return produtos_formatados.to_dict(orient='records')